### Load data & packages

### Packages

In [43]:
import numpy as np
import pandas as pd
import seaborn

### Data

In [66]:
# raw data
#path2019 = "~/Documents/UFABC/Disciplinas/Mineração de Dados/Dados para o Projeto/SINASC_2015-2019/ETLSINASC.DNRES_SP_2019_t.csv"
#path2018 = "~/Documents/UFABC/Disciplinas/Mineração de Dados/Dados para o Projeto/SINASC_2015-2019/ETLSINASC.DNRES_SP_2018_t.csv"
#path2017 = "~/Documents/UFABC/Disciplinas/Mineração de Dados/Dados para o Projeto/SINASC_2015-2019/ETLSINASC.DNRES_SP_2017_t.csv"
#path2016 = "~/Documents/UFABC/Disciplinas/Mineração de Dados/Dados para o Projeto/SINASC_2015-2019/ETLSINASC.DNRES_SP_2016_t.csv"
#path2015 = "~/Documents/UFABC/Disciplinas/Mineração de Dados/Dados para o Projeto/SINASC_2015-2019/ETLSINASC.DNRES_SP_2019_t.csv"

# filtered data
path2019 = "~/Documents/jupyter-workplace/data-mining-assignment/filtered_data2019.csv"
path2018 = "~/Documents/jupyter-workplace/data-mining-assignment/filtered_data2018.csv"
path2017 = "~/Documents/jupyter-workplace/data-mining-assignment/filtered_data2017.csv"
path2016 = "~/Documents/jupyter-workplace/data-mining-assignment/filtered_data2016.csv"
path2015 = "~/Documents/jupyter-workplace/data-mining-assignment/filtered_data2015.csv"

data2019 = pd.read_csv(path2019, index_col=0)
data2018 = pd.read_csv(path2018, index_col=0)
data2017 = pd.read_csv(path2017, index_col=0)
data2016 = pd.read_csv(path2016, index_col=0)
data2015 = pd.read_csv(path2015, index_col=0)

## Filter

### Filter variables

In [ ]:
# Idade Mae; Estado Civil; Escolaridade; Etnia/Cor; Municipio de Residencia; Local de nascimento; Ocupação
perfil_materno = ["IDADEMAE","ESTCIVMAE","ESCMAE","RACACOR","res_MUNNOME","def_loc_nasc","CODOCUPMAE"]

# Idade Gestacional; Consultas de Pre natal; Mês que iniciou o prenatal
perfil_gest <- ["GESTACAO","CONSULTAS","MESPRENAT"]

# APGAR; Peso ao nascimento; Anomalia congenita; prematuridade;
perfil_nasc <- ["APGAR1","APGAR5","PESO","IDANOMAL","parto_prematuro"]

filt <- perfil_materno+perfil_gest+perfil_nasc

### Filter for sampling

City == São Paulo

Age = 17 < x < 50

Anomalia == 2 [Não!]

In [67]:
import warnings
warnings.filterwarnings('ignore')

# criar uma função que filtre e junte os bancos de dados e retorne o dataframe!

# filter são paulo!
data2019_sp = data2019[data2019['res_MUNNOME'] == "São Paulo"]
data2018_sp = data2018[data2018['res_MUNNOME'] == "São Paulo"]
data2017_sp = data2017[data2017['res_MUNNOME'] == "São Paulo"]
data2016_sp = data2016[data2016['res_MUNNOME'] == "São Paulo"]
data2015_sp = data2015[data2015['res_MUNNOME'] == "São Paulo"]

# filter age (17 < x < 50)
data2019_sp = data2019_sp[(17 < data2019['IDADEMAE']) & (data2019['IDADEMAE'] < 50)]
data2018_sp = data2018_sp[(17 < data2018['IDADEMAE']) & (data2018['IDADEMAE'] < 50)]
data2017_sp = data2017_sp[(17 < data2017['IDADEMAE']) & (data2017['IDADEMAE'] < 50)]
data2016_sp = data2016_sp[(17 < data2016['IDADEMAE']) & (data2016['IDADEMAE'] < 50)]
data2015_sp = data2015_sp[(17 < data2015['IDADEMAE']) & (data2015['IDADEMAE'] < 50)]

# filter no anomalies
data2019_sp = data2019_sp[data2019['IDANOMAL'] == 2]
data2018_sp = data2018_sp[data2018['IDANOMAL'] == 2]
data2017_sp = data2017_sp[data2017['IDANOMAL'] == 2]
data2016_sp = data2016_sp[data2016['IDANOMAL'] == 2]
data2015_sp = data2015_sp[data2015['IDANOMAL'] == 2]

# filter ocupation?

Checking the filtered data...

In [68]:
print("2019\n\n")

print("Idade da mãe: \n",data2019_sp['IDADEMAE'].describe(),"\n")
print("Municipio: \n",data2019_sp['res_MUNNOME'].describe(),"\n")
print("Anomalia: \n",data2019_sp['IDANOMAL'].describe())

2019


Idade da mãe: 
 count    149399.000000
mean         29.111614
std           6.421685
min          18.000000
25%          24.000000
50%          29.000000
75%          34.000000
max          49.000000
Name: IDADEMAE, dtype: float64 

Municipio: 
 count        149399
unique            1
top       São Paulo
freq         149399
Name: res_MUNNOME, dtype: object 

Anomalia: 
 count    149399.0
mean          2.0
std           0.0
min           2.0
25%           2.0
50%           2.0
75%           2.0
max           2.0
Name: IDANOMAL, dtype: float64


In [69]:
print("2018\n\n")

print("Idade da mãe: \n",data2018_sp['IDADEMAE'].describe(),"\n")
print("Municipio: \n",data2018_sp['res_MUNNOME'].describe(),"\n")
print("Anomalia: \n",data2018_sp['IDANOMAL'].describe())

2018


Idade da mãe: 
 count    154498.000000
mean         29.018505
std           6.380069
min          18.000000
25%          24.000000
50%          29.000000
75%          34.000000
max          49.000000
Name: IDADEMAE, dtype: float64 

Municipio: 
 count        154498
unique            1
top       São Paulo
freq         154498
Name: res_MUNNOME, dtype: object 

Anomalia: 
 count    154498.0
mean          2.0
std           0.0
min           2.0
25%           2.0
50%           2.0
75%           2.0
max           2.0
Name: IDANOMAL, dtype: float64


In [70]:
print("2017\n\n")

print("Idade da mãe: \n",data2017_sp['IDADEMAE'].describe(),"\n")
print("Municipio: \n",data2017_sp['res_MUNNOME'].describe(),"\n")
print("Anomalia: \n",data2017_sp['IDANOMAL'].describe())

2017


Idade da mãe: 
 count    157228.000000
mean         28.862397
std           6.342752
min          18.000000
25%          24.000000
50%          29.000000
75%          34.000000
max          49.000000
Name: IDADEMAE, dtype: float64 

Municipio: 
 count        157228
unique            1
top       São Paulo
freq         157228
Name: res_MUNNOME, dtype: object 

Anomalia: 
 count    157228.0
mean          2.0
std           0.0
min           2.0
25%           2.0
50%           2.0
75%           2.0
max           2.0
Name: IDANOMAL, dtype: float64


In [71]:
print("2016\n\n")

print("Idade da mãe: \n",data2016_sp['IDADEMAE'].describe(),"\n")
print("Municipio: \n",data2016_sp['res_MUNNOME'].describe(),"\n")
print("Anomalia: \n",data2016_sp['IDANOMAL'].describe())

2016


Idade da mãe: 
 count    154788.000000
mean         28.697690
std           6.305601
min          18.000000
25%          23.000000
50%          29.000000
75%          34.000000
max          49.000000
Name: IDADEMAE, dtype: float64 

Municipio: 
 count        154788
unique            1
top       São Paulo
freq         154788
Name: res_MUNNOME, dtype: object 

Anomalia: 
 count    154788.0
mean          2.0
std           0.0
min           2.0
25%           2.0
50%           2.0
75%           2.0
max           2.0
Name: IDANOMAL, dtype: float64


In [72]:
print("2015\n\n")

print("Idade da mãe: \n",data2015_sp['IDADEMAE'].describe(),"\n")
print("Municipio: \n",data2015_sp['res_MUNNOME'].describe(),"\n")
print("Anomalia: \n",data2015_sp['IDANOMAL'].describe())

2015


Idade da mãe: 
 count    149399.000000
mean         29.111614
std           6.421685
min          18.000000
25%          24.000000
50%          29.000000
75%          34.000000
max          49.000000
Name: IDADEMAE, dtype: float64 

Municipio: 
 count        149399
unique            1
top       São Paulo
freq         149399
Name: res_MUNNOME, dtype: object 

Anomalia: 
 count    149399.0
mean          2.0
std           0.0
min           2.0
25%           2.0
50%           2.0
75%           2.0
max           2.0
Name: IDANOMAL, dtype: float64


## Process data

### Binding years

In [ ]:
# checking dimensions 

# how many 